In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.metrics import confusion_matrix
import time

In [2]:
def image_data_generator(data_folder, img_size=(224, 224), batch_size=32, color_mode='rgb'):
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    generator = datagen.flow_from_directory(
        data_folder,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',  # 변경된 부분
        color_mode=color_mode
    )
    return generator

In [3]:
def ResNet50_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)  # 변경된 부분
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', AUC(name='auc')])  # 변경된 부분
    return model

In [4]:
base_path = r'C:\Users\pc\Desktop\CNN\All_feature_multiclass_CNN'
train_data_folder = os.path.join(base_path, 'Train_Image')
val_data_folder = os.path.join(base_path, 'Validation_Image')
test_data_folder = os.path.join(base_path, 'Test_Image')

# 데이터 제너레이터 생성
train_generator = image_data_generator(train_data_folder, batch_size=128)
val_generator = image_data_generator(val_data_folder, batch_size=128)
test_generator = image_data_generator(test_data_folder, batch_size=128)

# 모델 정의 (클래스 개수 11로 설정)
model = ResNet50_model(input_shape=(224, 224, 3), num_classes=11)

# 배치 크기와 에포크 설정
epochs = 10

# 모델 훈련
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

# 테스트하기 전에 시간 측정 시작
start_time = time.time()

# 모델 평가
test_loss, test_accuracy, test_auc = model.evaluate(test_generator)

# 시간 측정 종료 및 결과 출력
end_time = time.time()
elapsed_time = end_time - start_time

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test AUC:", test_auc)
print(f"Test Duration: {elapsed_time:.4f} seconds")

Found 320000 images belonging to 11 classes.
Found 40000 images belonging to 11 classes.
Found 40000 images belonging to 11 classes.
Epoch 1/10
2500/2500 [==============================] - 583s 232ms/step - loss: 1.7762 - accuracy: 0.5042 - auc: 0.7906 - val_loss: 1.7395 - val_accuracy: 0.5076 - val_auc: 0.8011
Epoch 2/10
2500/2500 [==============================] - 473s 189ms/step - loss: 1.7306 - accuracy: 0.5089 - auc: 0.8052 - val_loss: 1.7236 - val_accuracy: 0.5100 - val_auc: 0.8091
Epoch 3/10
2500/2500 [==============================] - 474s 189ms/step - loss: 1.7165 - accuracy: 0.5102 - auc: 0.8107 - val_loss: 1.7109 - val_accuracy: 0.5113 - val_auc: 0.8139
Epoch 4/10
2500/2500 [==============================] - 474s 189ms/step - loss: 1.7051 - accuracy: 0.5108 - auc: 0.8147 - val_loss: 1.7136 - val_accuracy: 0.5106 - val_auc: 0.8152
Epoch 5/10
2500/2500 [==============================] - 474s 189ms/step - loss: 1.6965 - accuracy: 0.5113 - auc: 0.8177 - val_loss: 1.6953 - val_ac

In [5]:
from sklearn.metrics import confusion_matrix
import numpy as np

# 예측을 받습니다. 예측 결과는 각 클래스에 대한 확률을 반환합니다.
predictions = model.predict(test_generator, steps=len(test_generator))

# 각 예측에서 가장 큰 확률 값을 가진 인덱스를 찾아 해당 클래스로 설정합니다.
predicted_classes = np.argmax(predictions, axis=1)

# 실제 레이블을 얻습니다.
true_classes = test_generator.classes

# 테스트 제너레이터의 클래스 인덱스를 얻습니다.
# class_indices는 {'class_name': index} 형태의 딕셔너리입니다.
class_indices = test_generator.class_indices

# 클래스 인덱스를 이름에 매핑하기 위한 딕셔너리를 뒤집습니다.
# {index: 'class_name'} 형태로 만듭니다.
index_to_class = {v: k for k, v in class_indices.items()}

# 예측된 클래스 인덱스를 클래스 이름으로 변환합니다.
predicted_class_labels = [index_to_class[i] for i in predicted_classes]

# 실제 클래스 인덱스를 클래스 이름으로 변환합니다.
true_class_labels = [index_to_class[i] for i in true_classes]

# 혼동 행렬을 생성합니다.
conf_matrix = confusion_matrix(true_class_labels, predicted_class_labels, labels=list(index_to_class.values()))

print("Confusion Matrix:")
print(conf_matrix)

313/313 [==============================] - 52s 164ms/step
Confusion Matrix:
[[19617     2     0   165     0    31   117     1     0    65     2]
 [ 2203     0     0    15     0     1    16     0     0    12     0]
 [ 1440     0     0     7     0     0    12     0     0     4     0]
 [ 2538     0     0    25     0     3    19     0     0     8     0]
 [ 1434     0     0    12     0     6    15     0     0     6     0]
 [  274     0     0     4     0     1     3     0     0     3     0]
 [ 2517     0     0    24     0     2    26     1     0     9     0]
 [ 2303     0     0    19     0     3    15     0     0     9     1]
 [ 2010     0     0    18     0     4    17     0     0     4     3]
 [ 3077     0     0    30     0     6    16     0     0    10     0]
 [ 1777     0     0    14     0     4    10     0     0    10     0]]
